<a href="https://colab.research.google.com/github/masies/CRA/blob/main/pytorchConversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import torch
!pip3 install transformers
import transformers
!pip install sentencepiece
import sentencepiece as spm
import pandas as pd

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'

!gcloud config set project {project_id}
!gsutil ls

     |████████████████████████████████| 2.2MB 5.5MB/s 
     |████████████████████████████████| 3.3MB 29.6MB/s 
     |████████████████████████████████| 870kB 40.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=e825b85926dd36b02cbbb20a87f8398b92315848ee861448aed94a51a125bd58
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.7MB/s 
Updated property [core/project].
gs://code_review_automation/


In [2]:
model_number = 240000

# script for conversion in pythorch
!gsutil cp gs://{bucket_name}/pyTorch_coversion/code/tf_2_pytorch_T5.py ./tf_2_pytorch_T5.py

# Download the selected best model
!mkdir dumps
!mkdir current_model
!gsutil -m cp \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.data-00000-of-00002" \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.data-00001-of-00002" \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.index" \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.meta" \
  ./current_model/

# Download the configuration file
!gsutil cp gs://{bucket_name}/pyTorch_coversion/config/config.json ./current_model/config.json

# Convert the model
!python3 ./tf_2_pytorch_T5.py --tf_checkpoint_path ./current_model/model.ckpt-{model_number} --config_file ./current_model/config.json --pytorch_dump_path ./dumps

Copying gs://code_review_automation/pyTorch_coversion/code/tf_2_pytorch_T5.py...
- [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.data-00000-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.data-00001-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.index...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.meta...
| [4/4 files][126.0 MiB/126.0 MiB] 100% Done                                    
Operation completed over 4 objects/126.0 MiB.                                    
Co

In [3]:
# Download the validation script
!gsutil cp gs://{bucket_name}/pyTorch_coversion/code/validation.py ./validation.py
!mkdir core

!gsutil -m cp -r \
  "gs://code_review_automation/pyTorch_coversion/core/Logger.py" \
  ./core/

# Download the model and vocab
!gsutil cp gs://{bucket_name}/CodeReviewModel/TestModel.model ./current_model/TestModel.model
!gsutil cp gs://{bucket_name}/CodeReviewModel/TestModel.vocab ./current_model/TestModel.vocab

Copying gs://code_review_automation/pyTorch_coversion/code/validation.py...
/ [1 files][  6.1 KiB/  6.1 KiB]                                                
Operation completed over 1 objects/6.1 KiB.                                      
Copying gs://code_review_automation/pyTorch_coversion/core/Logger.py...
/ [1/1 files][   1016 B/   1016 B] 100% Done                                    
Operation completed over 1 objects/1016.0 B.                                     
Copying gs://code_review_automation/CodeReviewModel/TestModel.model...
- [1 files][762.0 KiB/762.0 KiB]                                                
Operation completed over 1 objects/762.0 KiB.                                    
Copying gs://code_review_automation/CodeReviewModel/TestModel.vocab...
/ [1 files][557.4 KiB/557.4 KiB]                                                
Operation completed over 1 objects/557.4 KiB.                                    
Copying gs://code_review_automation/dataset/old/fineTuning

In [ ]:
# Download the test set to get model accuracy on it
!gsutil cp gs://{bucket_name}/dataset/old/fineTuningDataset_v1/codeANDcomment_code/test.tsv ./data/codeANDcomment_code/test.tsv

# prepare source and target files
df = pd.read_csv("./data/codeANDcomment_code/test.tsv", sep='\t', names=["source","target"])

# initialize source and target files
f = open("./data/codeANDcomment_code/test.source", "w")
f.close()
f = open("./data/codeANDcomment_code/test.target", "w")


f.close()
with open("./data/codeANDcomment_code/test.source", "a") as source:
  with open("./data/codeANDcomment_code/test.target", "a") as target:
    for index, row in df.iterrows():
      source.write(row.source + "\n")
      target.write(row.target + "\n")

# run validation script
!python3 validation.py --beam_size 5 --batch_size 100 --data_dir /content/data/codeANDcomment_code --tokenizer_name ./current_model/TestModel.model --model_name_or_path ./dumps/pytorch_model.bin --config_name ./current_model/config.json

Copying gs://code_review_automation/dataset/old/fineTuningDataset_v1/codeANDcomment_code/test.tsv...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1621: FutureWarning: Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,
  0% 0/18 [00:00<?, ?it/s]